In [298]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
import tqdm

In [299]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [300]:
class QuantumData(Dataset):
    
    def __init__(self, x_file, y_file):
        self.data_x = pd.read_csv(x_file)
        self.data_y = pd.read_csv(y_file)
        
        self.data_x = torch.tensor(self.data_x.values, dtype=torch.float32)#Keeping float64 was throwing some kind of error
        self.data_y = torch.tensor(self.data_y.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X = self.data_x[idx]
        y = self.data_y[idx]

        return X, y

In [301]:
training_data = QuantumData('Dataset/Athens_x_train.csv', 'Dataset/Athens_y_train.csv')
testing_data = QuantumData('Dataset/Athens_x_test.csv', 'Dataset/Athens_y_test.csv')

In [302]:
print(pd.read_csv('Dataset/Athens_y_test.csv').shape)

(5352, 5)


In [303]:
train_loader = DataLoader(training_data, batch_size = 64 ,shuffle=True)
test_loader = DataLoader(testing_data, batch_size = 64,shuffle=True)

In [304]:
class Slot(nn.Module):

  def __init__(self):
    super(Slot, self).__init__()

    self.slot_fc1 = nn.Linear(1024,256)
    self.slot_fc2 = nn.Linear(256,128)
    #The slot's output array contains no(processor qubits) + 1 
    #The last class is to show no mapping
    #ith index is one in the last layer of the slot implies 
    #Each slot_i shows which processor qubit j it should be mapped to in its array
    self.slot_fc3 = nn.Linear(128,5)

  def forward(self,x):
    x = self.slot_fc1(x)
    x = F.relu(x)
    x = self.slot_fc2(x)
    x = F.relu(x)
    x = self.slot_fc3(x)
    x = F.relu(x)

    return x

In [305]:
class NeuralLayout(nn.Module):
    
    def __init__(self):
        super(NeuralLayout, self).__init__()

        self.dnn_fc1 = nn.Linear(78,264)
        self.dnn_fc2 = nn.Linear(264,1024)
        self.dnn_dropout = nn.Dropout(p=0.5)
        
        self.slots = nn.ModuleList([Slot() for i in range(5)])
        
    def forward(self, x):
      
        x = self.dnn_fc1(x)
        x = F.relu(x)
        x = self.dnn_fc2(x)
        x = F.relu(x)
        x = self.dnn_dropout(x)

        out = []

        for slot in self.slots:
          out.append(slot(x))

        return tuple(out)

In [306]:
nl = NeuralLayout().to(device=device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(nl.parameters(), lr=0.001)

In [307]:
num_epochs = 1
#print(len(list(enumerate(train_loader))))

In [317]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate((train_loader)):
        if targets.shape[0] != 64:
              break
        nl.train()

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        #print(data.shape)
        #print(targets.shape)
        

        # Forward
        cross_entropy_loss = nn.CrossEntropyLoss()

        out = nl(data)

        #print(targets[0])

        batch_target_tensor = torch.zeros(64,5,5).to(device=device)

        for batch_entry in range(64):
          target = targets[batch_entry]

          final_target_tensor = torch.zeros(1,5,5)

          for j,i in enumerate(target):
            batch_target_tensor[batch_entry,j,int(i)] = 1


          #print(target)
          #print(batch_tensor[batch_entry,:,:])
          #print()

        #Shape for batch_tensor should be 64 x 5 (slot) x 5 (for each processor qubit)


        loss = torch.tensor([0])
        for i,out_i in enumerate(out):
          #print(i,out_i)
          slot_i_loss = loss_fn(out_i,batch_target_tensor[:,i,:])
          if not torch.any(slot_i_loss.isnan()):
            loss += slot_i_loss
        #print(loss_fn(out_i,batch_target_tensor[:,i,:]))
        #print(loss)

        #loss = sum([loss_fn(out_i,batch_target_tensor[:,i,:]) for i,out_i in enumerate(out)])

        #print(out)
        
        # Backward
        optimizer.zero_grad()
        loss.backward(retain_graph=True)

        # Gradient descent or adam step
        optimizer.step()


RuntimeError: ignored

In [311]:
print([i for i in nl.parameters()])

[Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [310]:
for idx,(data,targets) in enumerate(test_loader):
    nl.eval()
    data = data.to(device=device)
    pred = nl(data)
    print(pred)

Streaming output truncated to the last 5000 lines.
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
    